In [1]:
library(lme4)
library(lmerTest)
library(sjPlot)

Loading required package: Matrix


Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

    lmer


The following object is masked from ‘package:stats’:

    step




# Load data

In [2]:
df <- read.csv("output/critical_long_format.csv")
#df <- read.csv("output/critical_long_format_with_dummy.csv")
df$ID <- df$client_id


# Data preparation

In [3]:
# center binary variables
df$critical_cent <- scale(df$critical, scale=F)
df$mail_cent <- scale(df$mailed, scale=F)
df$mailday_cent <- scale(df$mailday, scale=F)
df$weekday_cent <- scale(df$is_weekday, scale=F)


In [4]:
# factors for green hours, mail and mailday
df$critical_f <- as.factor(df$critical)
levels(df$critical_f) <- c("No green hours","Green hours (11am to 3pm)")
df$mail_f <- as.factor(df$mailed)
levels(df$mail_f) <- c("No mail","Green mail")
df$mail_day_f <- as.factor(df$mailday)
levels(df$mail_day_f) <- c("No mail for day","Mail for day")
df$weekday_f <- as.factor(df$is_weekday)
levels(df$weekday_f) <- c("Mo-Fr","Sa-Su")

In [5]:
df_maildays = df[df$mailday == 1,]
df_nomaildays = df[df$mailday == 0,]

# Analyzing start of chargers

## MailDays

### Model: `Starts ~ mail*critical + 1|D`

In [6]:
m1 <- glmer(charge_started ~ mail_cent*critical_cent+(1|ID),data=df_maildays,family = binomial(link = "logit"))
summary(m1)
#tab_model(m1)


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: charge_started ~ mail_cent * critical_cent + (1 | ID)
   Data: df_maildays

     AIC      BIC   logLik deviance df.resid 
  1000.3   1035.4   -495.1    990.3     8263 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.0797 -0.0684 -0.0240 -0.0205 15.9540 

Random effects:
 Groups Name        Variance Std.Dev.
 ID     (Intercept) 6.35     2.52    
Number of obs: 8268, groups:  ID, 318

Fixed effects:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -7.46295    0.57105 -13.069  < 2e-16 ***
mail_cent                1.52215    0.28451   5.350 8.80e-08 ***
critical_cent           -0.01644    0.38082  -0.043    0.966    
mail_cent:critical_cent  2.46177    0.54242   4.539 5.67e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr) ml_cnt cr

### Model: `Starts ~ mail*critical+weekday + 1|D`

In [7]:
m1 <- glmer(charge_started ~ mail_cent*critical_cent + weekday_cent+(1|ID),data=df_maildays,family = binomial(link = "logit"))

summary(m1)
#tab_model(m1)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: charge_started ~ mail_cent * critical_cent + weekday_cent + (1 |  
    ID)
   Data: df_maildays

     AIC      BIC   logLik deviance df.resid 
   999.4   1041.5   -493.7    987.4     8262 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.1840 -0.0721 -0.0229 -0.0190 16.0297 

Random effects:
 Groups Name        Variance Std.Dev.
 ID     (Intercept) 6.437    2.537   
Number of obs: 8268, groups:  ID, 318

Fixed effects:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -7.55056    0.58160 -12.982  < 2e-16 ***
mail_cent                1.52727    0.28458   5.367 8.01e-08 ***
critical_cent           -0.01538    0.38081  -0.040   0.9678    
weekday_cent            -0.35122    0.20518  -1.712   0.0869 .  
mail_cent:critical_cent  2.47065    0.54274   4.552 5.31e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0

### Model: `Starts ~ mail*critical*weekday + 1|D`

In [8]:
m1 <- glmer(charge_started ~ mail_cent*critical_cent*weekday_cent + (1|ID),data=df_maildays,family = binomial(link = "logit"))

summary(m1)

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.273994 (tol = 0.001, component 1)”


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: charge_started ~ mail_cent * critical_cent * weekday_cent + (1 |  
    ID)
   Data: df_maildays

     AIC      BIC   logLik deviance df.resid 
  1004.0   1067.2   -493.0    986.0     8259 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.2146 -0.0759 -0.0253 -0.0198 15.4711 

Random effects:
 Groups Name        Variance Std.Dev.
 ID     (Intercept) 5.754    2.399   
Number of obs: 8268, groups:  ID, 318

Fixed effects:
                                     Estimate Std. Error z value Pr(>|z|)    
(Intercept)                          -7.37816    0.52582 -14.032  < 2e-16 ***
mail_cent                             1.53001    0.30228   5.062 4.16e-07 ***
critical_cent                         0.05973    0.41092   0.145 0.884427    
weekday_cent                         -0.23930    0.38180  -0.627 0.530812    
mail_cent:critical_cent               2.

## No Mail Days

### Model: `Starts ~ critical*weekday + 1|ID`


In [9]:
m2 <- glmer(charge_started ~ critical_cent*weekday_cent+(1|ID),data=df_nomaildays,family = binomial(link = "logit"))
summary(m2)
#tab_model(m1)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: charge_started ~ critical_cent * weekday_cent + (1 | ID)
   Data: df_nomaildays

     AIC      BIC   logLik deviance df.resid 
  1170.9   1210.0   -580.5   1160.9    18439 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-0.4737 -0.0143 -0.0137 -0.0108 10.6114 

Random effects:
 Groups Name        Variance Std.Dev.
 ID     (Intercept) 14.25    3.774   
Number of obs: 18444, groups:  ID, 318

Fixed effects:
                           Estimate Std. Error z value Pr(>|z|)    
(Intercept)                -8.66678    0.77046 -11.249  < 2e-16 ***
critical_cent              -0.54188    0.20468  -2.647  0.00811 ** 
weekday_cent                0.04820    0.23752   0.203  0.83918    
critical_cent:weekday_cent -0.05845    0.47406  -0.123  0.90188    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
       

## ALL: Mail and No Mail Days

### Model: `Starts ~ mail*critical*weekday  + mailday_cent + 1|ID`

In [10]:
m4 <- glmer(charge_started ~ mail_cent*critical_cent*weekday_cent + mailday_cent +(1|ID),data=df,family = binomial(link = "logit"))
summary(m4)

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.162537 (tol = 0.001, component 1)”
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model is nearly unidentifiable: very large eigenvalue
 - Rescale variables?”


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
charge_started ~ mail_cent * critical_cent * weekday_cent + mailday_cent +  
    (1 | ID)
   Data: df

     AIC      BIC   logLik deviance df.resid 
  2189.0   2270.9  -1084.5   2169.0    26702 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.1569 -0.0700 -0.0288 -0.0229 21.2577 

Random effects:
 Groups Name        Variance Std.Dev.
 ID     (Intercept) 4.4      2.098   
Number of obs: 26712, groups:  ID, 318

Fixed effects:
                                       Estimate Std. Error  z value Pr(>|z|)
(Intercept)                          -6.8080668  0.0004363 -15605.5   <2e-16
mail_cent                             1.4858250  0.0004362   3406.1   <2e-16
critical_cent                        -0.1664508  0.0004362   -381.6   <2e-16
weekday_cent                          0.0533210  0.0004362    122.2   <2e-16
mailday_cent                         

### Model: `Starts ~ mailday*critical*weekday  + 1|ID`

In [11]:
m5 <- glmer(charge_started ~ mailday_cent*critical_cent*weekday_cent +(1|ID),data=df,family = binomial(link = "logit"))
summary(m5)
#tab_model(m4)

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.00503246 (tol = 0.001, component 1)”


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: charge_started ~ mailday_cent * critical_cent * weekday_cent +  
    (1 | ID)
   Data: df

     AIC      BIC   logLik deviance df.resid 
  2277.9   2351.6  -1129.9   2259.9    26703 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-0.7617 -0.0621 -0.0319 -0.0243 16.4105 

Random effects:
 Groups Name        Variance Std.Dev.
 ID     (Intercept) 3.929    1.982   
Number of obs: 26712, groups:  ID, 318

Fixed effects:
                                        Estimate Std. Error z value Pr(>|z|)
(Intercept)                             -6.56224    0.26852 -24.438  < 2e-16
mailday_cent                             0.73075    0.15700   4.654 3.25e-06
critical_cent                            0.02791    0.15775   0.177    0.860
weekday_cent                            -0.03868    0.17604  -0.220    0.826
mailday_cent:critical_cent               1.79219  

### Model: `Starts ~ mailday*critical*weekday  + 1|ID`

In [12]:
m6 <- glmer(charge_started ~ mailday_cent*critical_cent*weekday_cent + mail_cent +(1|ID),data=df,family = binomial(link = "logit"))
summary(m6)

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.165217 (tol = 0.001, component 1)”
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model is nearly unidentifiable: very large eigenvalue
 - Rescale variables?”


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: charge_started ~ mailday_cent * critical_cent * weekday_cent +  
    mail_cent + (1 | ID)
   Data: df

     AIC      BIC   logLik deviance df.resid 
  2210.2   2292.1  -1095.1   2190.2    26702 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.0294 -0.0689 -0.0296 -0.0234 25.0702 

Random effects:
 Groups Name        Variance Std.Dev.
 ID     (Intercept) 4.454    2.11    
Number of obs: 26712, groups:  ID, 318

Fixed effects:
                                          Estimate Std. Error   z value
(Intercept)                             -6.7653932  0.0004311 -15693.82
mailday_cent                            -0.4669626  0.0004310  -1083.33
critical_cent                            0.0737548  0.0004310    171.11
weekday_cent                            -0.0099800  0.0004310    -23.15
mail_cent                                1.8566810  0.0004310  

### Model: `Starts ~ mail*mailday*critical*weekday  + 1|ID`

In [13]:
m7 <- glmer(charge_started ~ mail_cent*mailday_cent*critical_cent*weekday_cent + (1|ID),data=df,family = binomial(link = "logit"))
summary(m7)

fixed-effect model matrix is rank deficient so dropping 4 columns / coefficients

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.698062 (tol = 0.001, component 1)”


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
charge_started ~ mail_cent * mailday_cent * critical_cent * weekday_cent +  
    (1 | ID)
   Data: df

     AIC      BIC   logLik deviance df.resid 
  2195.2   2301.7  -1084.6   2169.2    26699 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.1309 -0.0736 -0.0318 -0.0252 19.6986 

Random effects:
 Groups Name        Variance Std.Dev.
 ID     (Intercept) 3.588    1.894   
Number of obs: 26712, groups:  ID, 318

Fixed effects:
                                         Estimate Std. Error z value Pr(>|z|)
(Intercept)                             -6.631243   0.255259 -25.979  < 2e-16
mail_cent                                1.450836   0.293408   4.945 7.62e-07
mailday_cent                            -0.275140   0.263273  -1.045    0.296
critical_cent                           -0.148231   0.166089  -0.892    0.372
weekday_cent                    